# Plate modal analysis

This modal analysis of a plate works and draws off of a gmsh value to produce the modal analysis. This was confirmed by modeling the NAFEMS plate from the UK in 1990 and confirming the results

# Notes
- Gmsh must be installed and has been added to the docker file
- Only really works in dolfin not dolfinx. this is due to the lack of a good assembly with the boundary conditions needed for the problem

In [3]:
# import stuff


from dolfin import *
import dolfin
comm = dolfin.cpp.MPI.comm_world
rank = comm.Get_rank()
size = comm.Get_size()
print(f"rank: {rank}; size:{size}")
import petsc4py
import numpy as np
import time
import matplotlib.pyplot as plt
# %matplotlib notebook
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["optimize"] = True

rank: 0; size:1


In [5]:


# load in gmsh
mesh = Mesh(comm,"../mesh/extruded_box_panel.xml")
# # In[2]:

E, nu = Constant(200e9), Constant(0.3)
rho = Constant(8000)


In [6]:

# Lame coefficient for constitutive relation
mu = E/2./(1+nu)
lmbda = E*nu/(1+nu)/(1-2*nu)

def eps(v):
    return sym(grad(v))

def sigma(v):
    dim = v.geometric_dimension()
    return 2.0*mu*eps(v) + lmbda*tr(eps(v))*Identity(dim)



In [7]:

V = VectorFunctionSpace(mesh, 'Lagrange', degree=1)
u_ = TrialFunction(V)
du = TestFunction(V)

## Notes on Boundary Condition

The simply supported boundary condition (Z =0) is applied to the third subspace of the function space. V.sub(2)

In [8]:

def simply_supported(x,on_boundary):
    return on_boundary and (near(x[0],0.0) | near(x[0],1.0)| near(x[1],0.0)| near(x[1],1.0))



bc = DirichletBC(V.sub(2),Constant((0.)), simply_supported)



## Notes on building system
There are known symmetry issues with building the system with the K matrix. This solves them for uknown reasons

In [9]:


k_form = inner(sigma(du),eps(u_))*dx
l_form = Constant(1.)*u_[0]*dx
K = PETScMatrix(comm)
b = PETScVector(comm)
assemble_system(k_form, l_form,bc, A_tensor=K, b_tensor=b)


(<dolfin.cpp.la.PETScMatrix at 0x7f40d506bb48>,
 <dolfin.cpp.la.PETScVector at 0x7f40d506bba0>)

In [10]:

m_form = rho*dot(du,u_)*dx
M = PETScMatrix(comm)
assemble(m_form, tensor=M)
bc.zero(M)


In [11]:


eigensolver = SLEPcEigenSolver(K,M)
eigensolver.parameters['problem_type'] = 'gen_hermitian'
eigensolver.parameters['spectral_transform'] = 'shift-and-invert'
eigensolver.parameters['spectral_shift'] = 600.


In [ ]:

N_eig = 24   # number of eigenvalues
print("Computing {} first eigenvalues...".format(N_eig))
start = time.perf_counter()
eigensolver.solve(N_eig)
print(f"elapsed time: {time.perf_counter()-start}")
print(f"N Solutions Found: {eigensolver.get_number_converged()}")


In [13]:
print(f"N Solutions Found: {eigensolver.get_number_converged()}")


N Solutions Found: 77


In [14]:

if rank == 0:
    for i in range(N_eig):
        # Extract eigenpair
        r, c, rx, cx = eigensolver.get_eigenpair(i)
        # 3D eigenfrequency
        freq_2D = sqrt(r)/2/pi
        print(f"Solid FEM: {freq_2D:.3f} Hz")



Solid FEM: 0.001 Hz
Solid FEM: 0.001 Hz
Solid FEM: 0.001 Hz
Solid FEM: 109.679 Hz
Solid FEM: 274.220 Hz
Solid FEM: 274.497 Hz
Solid FEM: 436.502 Hz
Solid FEM: 548.356 Hz
Solid FEM: 548.975 Hz
Solid FEM: 708.213 Hz
Solid FEM: 709.139 Hz
Solid FEM: 929.196 Hz
Solid FEM: 929.296 Hz
Solid FEM: 977.005 Hz
Solid FEM: 1087.127 Hz
Solid FEM: 1087.742 Hz
Solid FEM: 1351.279 Hz
Solid FEM: 1352.463 Hz
Solid FEM: 1411.914 Hz
Solid FEM: 1414.467 Hz
Solid FEM: 1569.766 Hz
Solid FEM: 1570.548 Hz
Solid FEM: 1720.312 Hz
Solid FEM: 1830.968 Hz
Solid FEM: 1831.865 Hz
Solid FEM: 1936.246 Hz
Solid FEM: 1999.018 Hz
Solid FEM: 1999.827 Hz
Solid FEM: 2064.431 Hz
Solid FEM: 2064.502 Hz
Solid FEM: 2151.499 Hz
Solid FEM: 2153.339 Hz
Solid FEM: 2192.775 Hz
Solid FEM: 2193.382 Hz
Solid FEM: 2193.928 Hz
Solid FEM: 2407.303 Hz
Solid FEM: 2411.714 Hz
Solid FEM: 2497.263 Hz
Solid FEM: 2662.759 Hz
Solid FEM: 2681.604 Hz
Solid FEM: 2682.158 Hz
Solid FEM: 2764.337 Hz
Solid FEM: 2768.154 Hz
Solid FEM: 2831.266 Hz
Solid FE

In [ ]:
eigenmodes = []

for i in range(N_eig):
    # Extract eigenpair
    r, c, rx, cx = eigensolver.get_eigenpair(i)

    # 3D eigenfrequency
    freq_3D = sqrt(r)/2/pi

   
    # Initialize function and assign eigenvector
    eigenmode = Function(V,name="Eigenvector "+str(i))
    eigenmode.vector()[:] = rx

    eigenmodes.append(eigenmode)
    vedo.dolfin.plot(eigenmode,box,offscreen=True)
    vedo.screenshot(filename=f"../data/NAFEMS_PLATE_MODE_{i}_{freq_3D:.2f}_Hz")

#     file_results.write(eigenmode,0.)
    

In [ ]:
# visualie the data by exporting to file


# file_results = XDMFFile("../data/modal_analysis.xdmf")
# file_results.parameters["flush_output"] = True
# file_results.parameters["functions_share_mesh"] = True
# file_results.write(eigenmode, 0.)


# Visualization
## Notes
- Ne3eds to have vedo installed

In [ ]:
import pyvista as pv

pv.start_xvfb()

In [ ]:
from vedo.dolfin import *
from vedo import Box
import vedo
vedo.embedWindow(False)

In [ ]:
# add a frame box
box = Box(length=1, width=1, height=1).pos(0.5,0,0).wireframe()

In [ ]:
# iterate over the modes nadp lot them

for idx,mode in enumerate(eigenmodes):
        
    vedo.dolfin.plot(mode,box,offscreen=True)
    vedo.screenshot(filename=f"../data/NAFEMS_PLATE_MODE_{idx}")
